In [1]:
# Imports

import pandas as pd


In [2]:
# Read and clean the data we will merge

flux_df = pd.read_pickle("data/flux_df.pkl")
min_max_df = pd.read_pickle("data/min_max_df.pkl")
avg_df = pd.read_pickle("data/avg_df.pkl")

# flux_df has generic column names, let's fix that
flux_df = flux_df.rename(
    columns=lambda column: column + "_flux" if column != "day" else column
)

# This gives each observation a unique key which makes merging the dataframes easier
def get_id_column(df):
    return flux_df.index + "_" + flux_df["day"].astype(str)


flux_df["id"] = get_id_column(flux_df)
min_max_df["id"] = get_id_column(min_max_df)
avg_df["id"] = get_id_column(avg_df)



In [3]:
# Merge the data
df_merged = pd.merge(flux_df, min_max_df, on=["id", "day"])
df_merged = df_merged.merge(avg_df, on=["id", "day"])

# let's set df_merged's index to also be the neighbourhood of the observation
# this will help with handling the data later
df_merged.index = flux_df.index

# our "day" variable is an independent variable, let's move it back to the left
column_to_move = df_merged.pop("day")
df_merged.insert(0, "day", column_to_move)

In [4]:
# let's add the day of the week as data by using our "day" variable
def get_week_day(day):
    day = day % 7
    # 01/10/23 was a Sunday, therefore day 0 is Sunday, day 1 Monday and so forth.
    return (
        "sunday"
        if day == 0
        else "monday"
        if day == 1
        else "tuesday"
        if day == 2
        else "wednesday"
        if day == 3
        else "thursday"
        if day == 4
        else "friday"
        if day == 5
        else "saturday"
    )

# using .insert to have the column on the left
df_merged.insert(0, "week_day", df_merged["day"].apply(get_week_day))

In [5]:
# Now let's merge our google maps data
df_gmaps = pd.read_pickle("data/clean_gmaps.pkl")

def merge_gmaps_data():
    # Since in both our merged df and the gmaps df, the neighbourhoods are indexed in
    # alphabetical order, just doing this matches them perfectly
    for column in df_gmaps.columns:
        df_merged[column] = df_gmaps[column]

# This adds a lot of data that we can use, not all of which is necessarily useful
merge_gmaps_data()

display(df_merged)

,week_day,day,bicycle_flux,cargo_bicycle_flux,moped_flux,scooter_flux,id,bicycle_min,bicycle_max,bicycle_diff,...,nearest_metro_station_name_bicycling,nearest_metro_station_distance_bicycling,nearest_train_station_name_driving,nearest_train_station_distance_driving,nearest_metro_station_name_driving,nearest_metro_station_distance_driving,nearest_train_station_name_transit,nearest_train_station_distance_transit,nearest_metro_station_name_transit,nearest_metro_station_distance_transit
Aetsveld/Oostelijke Vechtoever,sunday,0,0.0,0.0,0.0,0.0,Aetsveld/Oostelijke Vechtoever_0,0.0,0.0,0.0,...,Gaasperplas,8031,Amsterdam Bijlmer ArenA Station,11934,Gaasperplas,7997,Amsterdam Bijlmer ArenA Station,13299,Gaasperplas,8753.0
Amstel III/Bullewijk,sunday,0,0.0,0.0,36.0,0.0,Amstel III/Bullewijk_0,0.0,0.0,0.0,...,Station Holendrecht,560,Amsterdam Holendrecht Station,695,Station Holendrecht,941,Amsterdam Holendrecht Station,719,Station Holendrecht,560.0
Amsterdamse Poort e.o.,sunday,0,0.0,1.0,17.0,0.0,Amsterdamse Poort e.o._0,0.0,0.0,0.0,...,Station Bijlmer ArenA,411,Amsterdam Bijlmer ArenA Station,671,Station Bijlmer ArenA,671,Amsterdam Bijlmer ArenA Station,378,Station Bijlmer ArenA,378.0
Apollobuurt,sunday,0,7.0,0.0,46.0,0.0,Apollobuurt_0,2.0,5.0,3.0,...,Station Zuid,1343,Amsterdam Zuid Station,1634,De Pijp,2145,Amsterdam Zuid Station,1902,Station Zuid,1253.0
Banne Buiksloot,sunday,0,0.0,0.0,0.0,0.0,Banne Buiksloot_0,0.0,0.0,0.0,...,Noorderpark,1633,Amsterdam Sloterdijk Station,10657,Noorderpark,1881,Amsterdam Centraal Station,4919,Noorderpark,1436.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Westlandgracht,wednesday,31,5.0,0.0,1.0,0.0,Westlandgracht_31,14.0,19.0,5.0,...,Henk Sneevlietweg,628,Amsterdam Lelylaan Station,1392,Henk Sneevlietweg,812,Amsterdam Lelylaan Station,1979,Henk Sneevlietweg,619.0
Willemspark,wednesday,31,0.0,0.0,0.0,0.0,Willemspark_31,0.0,0.0,0.0,...,Amstelveenseweg,1977,Amsterdam Lelylaan Station,2909,Heemstedestraat,2345,Amsterdam Lelylaan Station,2479,Amstelveenseweg,2084.0
Zeeburgereiland/Bovendiep,wednesday,31,0.0,0.0,0.0,0.0,Zeeburgereiland/Bovendiep_31,1.0,1.0,0.0,...,Verrijn Stuartweg,5595,Amsterdam Muiderpoort Station,3977,Weesperplein,5785,Amsterdam Muiderpoort Station,2685,Wibautstraat,4889.0
Zuid Pijp,wednesday,31,0.0,0.0,1.0,0.0,Zuid Pijp_31,2.0,2.0,0.0,...,De Pijp,0,Amsterdam RAI Station,2404,De Pijp,0,Amsterdam Muiderpoort Station,2610,Wibautstraat,1275.0


In [10]:
# Below code merges population data
ams_data = pd.read_csv("data/amsterdam_data.csv")

# Prepare ams_data for merging
ams_data.set_index('Name', inplace=True)
ams_data = ams_data.rename_axis(None)
ams_data = ams_data.drop('Unnamed: 0', axis=1, errors='ignore')

# Loop through the index of df_merged
for index in df_merged.index:
    # Check if the index is present in ams_data
    if index in ams_data.index:
        # Get the corresponding row in ams_data using the index
        ams_row = ams_data.loc[index]
        
        # Extract the population number from the 'Population' column
        population_number = ams_row['Population']
        
        # Update the 'Population' column in df_merged
        df_merged.at[index, 'population'] = population_number

        # Loop through age group columns in ams_data
        age_columns = ['0 - 3 years (%)', '4 - 12 years (%)', '13 - 17 years (%)', '18 - 26 years (%)', '27 - 65 years (%)', '66 + years (%)']
        highest_percentage_age_column = max(age_columns, key=lambda col: ams_row[col])

        # Update the 'highest_percentage_age' column in df_merged
        df_merged.at[index, 'highest_percentage_age'] = highest_percentage_age_column

        # Extract income data
        income_number = ams_row['Predicted mean disposable household income']

        # Add to income column
        df_merged.at[index, 'predicted_mean_disposable_household_income'] = income_number
        

# Save df_merged into a pickle file
df_merged.to_pickle("regression_data.pkl")

display(df_merged)


,week_day,day,bicycle_flux,cargo_bicycle_flux,moped_flux,scooter_flux,id,bicycle_min,bicycle_max,bicycle_diff,...,nearest_train_station_distance_driving,nearest_metro_station_name_driving,nearest_metro_station_distance_driving,nearest_train_station_name_transit,nearest_train_station_distance_transit,nearest_metro_station_name_transit,nearest_metro_station_distance_transit,population,highest_percentage_age,predicted_mean_disposable_household_income
Aetsveld/Oostelijke Vechtoever,sunday,0,0.0,0.0,0.0,0.0,Aetsveld/Oostelijke Vechtoever_0,0.0,0.0,0.0,...,11934,Gaasperplas,7997,Amsterdam Bijlmer ArenA Station,13299,Gaasperplas,8753.0,4402.0,27 - 65 years (%),39520.1
Amstel III/Bullewijk,sunday,0,0.0,0.0,36.0,0.0,Amstel III/Bullewijk_0,0.0,0.0,0.0,...,695,Station Holendrecht,941,Amsterdam Holendrecht Station,719,Station Holendrecht,560.0,3546.0,18 - 26 years (%),39520.1
Amsterdamse Poort e.o.,sunday,0,0.0,1.0,17.0,0.0,Amsterdamse Poort e.o._0,0.0,0.0,0.0,...,671,Station Bijlmer ArenA,671,Amsterdam Bijlmer ArenA Station,378,Station Bijlmer ArenA,378.0,8272.0,27 - 65 years (%),39520.1
Apollobuurt,sunday,0,7.0,0.0,46.0,0.0,Apollobuurt_0,2.0,5.0,3.0,...,1634,De Pijp,2145,Amsterdam Zuid Station,1902,Station Zuid,1253.0,8825.0,27 - 65 years (%),88559.5
Banne Buiksloot,sunday,0,0.0,0.0,0.0,0.0,Banne Buiksloot_0,0.0,0.0,0.0,...,10657,Noorderpark,1881,Amsterdam Centraal Station,4919,Noorderpark,1436.0,14806.0,27 - 65 years (%),42139.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Westlandgracht,wednesday,31,5.0,0.0,1.0,0.0,Westlandgracht_31,14.0,19.0,5.0,...,1392,Henk Sneevlietweg,812,Amsterdam Lelylaan Station,1979,Henk Sneevlietweg,619.0,13194.0,27 - 65 years (%),46335.9
Willemspark,wednesday,31,0.0,0.0,0.0,0.0,Willemspark_31,0.0,0.0,0.0,...,2909,Heemstedestraat,2345,Amsterdam Lelylaan Station,2479,Amstelveenseweg,2084.0,5561.0,27 - 65 years (%),89532.3
Zeeburgereiland/Bovendiep,wednesday,31,0.0,0.0,0.0,0.0,Zeeburgereiland/Bovendiep_31,1.0,1.0,0.0,...,3977,Weesperplein,5785,Amsterdam Muiderpoort Station,2685,Wibautstraat,4889.0,5679.0,27 - 65 years (%),42204.7
Zuid Pijp,wednesday,31,0.0,0.0,1.0,0.0,Zuid Pijp_31,2.0,2.0,0.0,...,2404,De Pijp,0,Amsterdam Muiderpoort Station,2610,Wibautstraat,1275.0,7917.0,27 - 65 years (%),39976.9
